# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [107]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [108]:
city_df = pd.read_csv("output_data/cities.csv")
city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Clouds,Wind,Country,Date
0,0,qaanaaq,77.4840,-69.3632,49.15,47,100,6.13,GL,1624481597
1,1,hilo,19.7297,-155.0900,88.29,79,28,3.00,US,1624481597
2,2,anuradhapura,8.3565,80.4173,77.02,80,61,13.13,LK,1624481598
3,3,plettenberg bay,-34.0527,23.3716,56.19,83,100,7.18,ZA,1624481598
4,4,punta arenas,-53.1500,-70.9167,41.02,93,90,6.91,CL,1624481353


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [109]:
 gmaps.configure(g_key)

In [110]:
location = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"]

In [111]:
fig = gmaps.figure()

heat_map = gmaps.heatmap_layer(
    location, 
    weights=city_df["Humidity"],
    dissipating=False,
    max_intensity=200,
    point_radius = 3)
fig.add_layer(heat_map)
print(fig)


Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [112]:
perfect_city_df = city_df.loc[(city_df["Wind"]<= 10) & (city_df["Clouds"] <= 0) & 
                                (city_df["Max Temperature"]>= 70) & (city_df["Max Temperature"]<= 80)].dropna()

perfect_city_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Clouds,Wind,Country,Date
32,32,gat,31.6100,34.7642,76.82,77,0,3.62,IL,1624481608
101,101,chaman,30.9236,66.4512,77.04,9,0,3.56,PK,1624481632
141,141,alexandria,31.2156,29.9553,78.76,69,0,6.91,EG,1624481622
259,259,sarh,9.1429,18.3923,75.11,88,0,0.00,TD,1624481692
362,362,khash,28.2211,61.2158,76.71,14,0,5.55,IR,1624481739
375,375,komsomolskiy,40.4272,71.7189,72.32,33,0,6.91,UZ,1624481744
436,436,marneuli,41.4759,44.8089,79.54,28,0,4.41,GE,1624481766
452,452,korla,41.7597,86.1469,72.55,23,0,7.52,CN,1624481772
476,476,birjand,32.8663,59.2211,72.07,14,0,9.22,IR,1624481781


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [113]:
hotel_df = perfect_city_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
32,gat,IL,31.6100,34.7642,
101,chaman,PK,30.9236,66.4512,
141,alexandria,EG,31.2156,29.9553,
259,sarh,TD,9.1429,18.3923,
362,khash,IR,28.2211,61.2158,
375,komsomolskiy,UZ,40.4272,71.7189,
436,marneuli,GE,41.4759,44.8089,
452,korla,CN,41.7597,86.1469,
476,birjand,IR,32.8663,59.2211,


In [114]:
hotels = []

for city in range(len(hotel_df["City"])):

    lat = row.Latitude
    lng = row.Longitude

    #city_coordinate = f"{lat},{lng}"

    params = {
        "location": hotel_df, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
32,gat,IL,31.6100,34.7642,Nearest hotel not found
101,chaman,PK,30.9236,66.4512,Nearest hotel not found
141,alexandria,EG,31.2156,29.9553,Nearest hotel not found
259,sarh,TD,9.1429,18.3923,Nearest hotel not found
362,khash,IR,28.2211,61.2158,Nearest hotel not found
375,komsomolskiy,UZ,40.4272,71.7189,Nearest hotel not found
436,marneuli,GE,41.4759,44.8089,Nearest hotel not found
452,korla,CN,41.7597,86.1469,Nearest hotel not found
476,birjand,IR,32.8663,59.2211,Nearest hotel not found


In [115]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [116]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(gmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))